In [ ]:
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [ ]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from pygad.kerasga import model_weights_as_vector,model_weights_as_matrix
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(solution):
    return (env.play(np.array(solution))[0],)

def cx_function(parent1,parent2,alpha):
    child = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    return creator.Individual(child)

def individual_gain(solution):
    res = env.play(np.array(solution))
    return res[1]-res[2]

In [ ]:
def steady_state(pop_size,n_offspring,tournsize_parent,cxpb,alpha,mutpb,sigma,indpb,tournsize_survival,ngen,verbose):
    
    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    
    fitnesses = toolbox.map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    hof.update(pop)
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    record = stats.compile(pop) if stats else {}
    logbook.record(gen=0, nevals=len(pop), **record)
    if verbose==1:
        print(logbook.stream)
    
    for g in range(ngen):
    
        # Select parents
        parents = tools.selTournament(individuals=pop,k=2*n_offspring*pop_size,tournsize=tournsize_parent)

        # Generate offspring with crossover
        offspring = []
        for parent1, parent2 in zip(parents[::2], parents[1::2]):
            if np.random.uniform() < cxpb:
                if alpha=="random":
                    child = cx_function(parent1, parent2,np.random.uniform())
                else:
                    child = cx_function(parent1, parent2,alpha)
            else:
                child = random.sample([parent1,parent2],1)[0]
            offspring.append(child)

        # Apply mutation on the offspring
        for mutant in offspring:
            if np.random.uniform() < mutpb:
                if indpb=="random":
                    tools.mutGaussian(mutant,mu=0,sigma=sigma,indpb=np.random.uniform())
                else:
                    tools.mutGaussian(mutant,mu=0,sigma=sigma,indpb=indpb)
                

        # Evaluate the offspring
        fitnesses = toolbox.map(toolbox.evaluate, offspring)
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit
            
        # Add offspring to the population
        pop = pop + offspring
        hof.update(pop)
        # Survival Selection
        pop = tools.selTournament(individuals=pop,k=pop_size,tournsize=tournsize_survival)

        record = stats.compile(pop) if stats else {}
        logbook.record(gen=g+1, nevals=len(pop), **record)
        if verbose==1:
            print(logbook.stream)
    
    return hof, logbook

In [ ]:
def generational_model(pop_size,n_offspring,tournsize_parent,cxpb,alpha,mutpb,sigma,indpb,tournsize_survival,ngen,verbose):
    
    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    
    fitnesses = toolbox.map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    hof.update(pop)
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    record = stats.compile(pop) if stats else {}
    logbook.record(gen=0, nevals=len(pop), **record)
    if verbose==1:
        print(logbook.stream)
    
    for g in range(ngen):
    
        # Select parents
        parents = tools.selTournament(individuals=pop,k=n_offspring*2*pop_size,tournsize=tournsize_parent)

        # Generate offspring with crossover
        offspring = []
        for parent1, parent2 in zip(parents[::2], parents[1::2]):
            if np.random.uniform() < cxpb:
                if alpha=="random":
                    child = cx_function(parent1, parent2,np.random.uniform())
                else:
                    child = cx_function(parent1, parent2,alpha)
            else:
                child = random.sample([parent1,parent2],1)[0]
            offspring.append(child)

        # Apply mutation on the offspring
        for mutant in offspring:
            if np.random.uniform() < mutpb:
                if indpb=="random":
                    tools.mutGaussian(mutant,mu=0,sigma=sigma,indpb=np.random.uniform())
                else:
                    tools.mutGaussian(mutant,mu=0,sigma=sigma,indpb=indpb)

        # Evaluate the offspring
        fitnesses = toolbox.map(toolbox.evaluate, offspring)
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit
        
        offspring = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)
        pop = offspring
        hof.update(pop)
        
        record = stats.compile(pop) if stats else {}
        logbook.record(gen=g+1, nevals=len(pop), **record)
        if verbose==1:
            print(logbook.stream)
    
    return hof, logbook

In [ ]:
def simulation(enemy_no,algorithm,n_simulation,hidden_neurons,params):
    
    individual_size = parameter_count(hidden_neurons)
    
    headless = True
    if headless:
        os.environ["SDL_VIDEODRIVER"] = "dummy"
    
    global env,toolbox,stats
    env = Environment(playermode="ai",
                      player_controller=player_controller(hidden_neurons),
                      speed="fastest",
                      enemymode="static",
                      enemies=enemy_no,
                      level=2,
                      visuals=False,
                      logs="off")
    
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-1,1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)
    pool = multiprocessing.Pool()
    toolbox.register("map", pool.map)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("Mean", np.mean)
    stats.register("Max", np.max)
    stats.register("Std", np.std)
    
    results = []
    if algorithm=="steady_state":
        for iteration in range(n_simulation): 
            hof,logs = steady_state(**params)
            for i in range(len(logs)):
                logs[i]['iteration']=iteration
                logs[i]['enemy']=enemy_no
                logs[i]['algorithm']=algorithm
                logs[i]['hof'] = hof
            results = results + logs
            
    elif algorithm=="generational_model":
        for iteration in range(n_simulation): 
            hof,logs = generational_model(**params)
            for i in range(len(logs)):
                logs[i]['iteration']=iteration
                logs[i]['enemy']=enemy_no
                logs[i]['algorithm']=algorithm
                logs[i]['hof'] = hof
            results = results + logs
            
    return pd.DataFrame(results)

In [ ]:
def objective(trial):
    
    params = {'pop_size': trial.suggest_int("pop_size", 100,100,step=100),
              'n_offspring': trial.suggest_int("n_offspring", 1,3,step=1),
              'tournsize_parent': trial.suggest_int("tournsize_parent", 1, 3, step=1),
              'cxpb': trial.suggest_float("cxpb", 0.5, 1., step=0.1),
              'alpha': trial.suggest_categorical("alpha",["random",0.5]),     
              'mutpb': trial.suggest_float("mutpb", 0.1, 0.5, step=0.1),
              'sigma': trial.suggest_float("sigma", 0.5, 2.5, step=0.5),
              'indpb': trial.suggest_categorical("indpb",["random",0.01,0.1,0.2]),
              'tournsize_survival': trial.suggest_int("tournsize_survival", 1, 3, step=1),
              'ngen':20,'verbose':1}
    
    hidden_neurons = 10
    enemy_no = "5"
    algorithm = "steady_state"
    n_simulation = 3
    
    logs_pd = simulation(enemy_no,algorithm,n_simulation,hidden_neurons,params)
    
    return np.mean(logs_pd['Max'])


study = optuna.create_study(study_name="steady_state", direction="maximize")

study.optimize(objective, n_trials=100)

In [ ]:
simulation_loop = []
simulation_loop.append({'enemy':"5",'algorithm':'steady_state','params':{'pop_size': 100,
                                                                       'n_offspring': 2,
                                                                       'tournsize_parent': 3,
                                                                       'cxpb': 0.9,
                                                                       'alpha': 'random',
                                                                       'mutpb': 0.3,
                                                                       'sigma': 2.0,
                                                                       'indpb': 0.1,
                                                                       'tournsize_survival': 3,
                                                                       'ngen':30,
                                                                       'verbose':1}})
simulation_loop.append({'enemy':"6",'algorithm':'steady_state','params':{'pop_size': 100,
                                                                       'n_offspring': 3,
                                                                       'tournsize_parent': 2,
                                                                       'cxpb': 0.5,
                                                                       'alpha': 0.5,
                                                                       'mutpb': 0.3,
                                                                       'sigma': 0.5,
                                                                       'indpb': 0.01,
                                                                       'tournsize_survival': 3,
                                                                       'ngen':30,
                                                                       'verbose':1}})
simulation_loop.append({'enemy':"7",'algorithm':'steady_state','params':{'pop_size': 100,
                                                                       'n_offspring': 2,
                                                                       'tournsize_parent': 2,
                                                                       'cxpb': 0.9,
                                                                       'alpha': 'random',
                                                                       'mutpb': 0.5,
                                                                       'sigma': 0.5,
                                                                       'indpb': 0.1,
                                                                       'tournsize_survival': 3,
                                                                       'ngen':30,
                                                                       'verbose':1}})
simulation_loop.append({'enemy':"5",'algorithm':'generational_model','params':{'pop_size': 100,
                                                                             'n_offspring': 3,
                                                                             'tournsize_parent': 2,
                                                                             'cxpb': 0.7,
                                                                             'alpha': 'random',
                                                                             'mutpb': 0.1,
                                                                             'sigma': 0.5,
                                                                             'indpb': 'random',
                                                                             'tournsize_survival': 3,
                                                                             'ngen':30,
                                                                             'verbose':1}})
simulation_loop.append({'enemy':"6",'algorithm':'generational_model','params':{'pop_size': 100,
                                                                             'n_offspring': 2,
                                                                             'tournsize_parent': 3,
                                                                             'cxpb': 0.9,
                                                                             'alpha': 'random',
                                                                             'mutpb': 0.2,
                                                                             'sigma': 1.0,
                                                                             'indpb': 0.1,
                                                                             'tournsize_survival': 3,
                                                                             'ngen':30,
                                                                             'verbose':1}})
simulation_loop.append({'enemy':"7",'algorithm':'generational_model','params':{'pop_size': 100,
                                                                             'n_offspring': 3,
                                                                             'tournsize_parent': 3,
                                                                             'cxpb': 0.7,
                                                                             'alpha': 'random',
                                                                             'mutpb': 0.5,
                                                                             'sigma': 2.5,
                                                                             'indpb': 0.01,
                                                                             'tournsize_survival': 3,
                                                                             'ngen':30,
                                                                             'verbose':1}})

In [ ]:
hidden_neurons = 10
n_simulation = 10

results = []
for s in simulation_loop:
    results.append(simulation(s['enemy'],s['algorithm'],n_simulation,hidden_neurons,s['params']))
    
results_pd = pd.concat(results)
results_pd['Algorithm'] = 'Steady State Model'
results_pd.loc[results_pd['algorithm']=='generational_model','Algorithm'] = 'Generational Model'

In [ ]:
results_pd = pd.read_excel("/kaggle/input/evoman-results/results_v2.xlsx",index_col=0).reset_index()
results_pd['Algorithm'] = 'Steady State Model'
results_pd.loc[results_pd['algorithm']=='generational_model','Algorithm'] = 'Generational Model'

In [ ]:
plt.figure(figsize=(12,6))
enemy = "7"
logs_pd = results_pd[results_pd['enemy']==int(enemy)]
sns.lineplot(data=logs_pd, x='gen', y='Mean', estimator='mean',hue='Algorithm', errorbar=('ci', 95), linestyle='dashed')
sns.lineplot(data=logs_pd, x='gen', y='Max', estimator='mean',hue='Algorithm', errorbar=('ci', 95))
plt.xlabel('Generation')
plt.ylabel('Average Fitness')
plt.title('Comparision Between Average Fitness - Enemy '+str(enemy))
plt.savefig('enemy_'+enemy+'.png')

In [ ]:

for s in simulation_loop:
    hidden_neurons = 10
    individual_size = parameter_count(hidden_neurons)

    headless = True
    if headless:
        os.environ["SDL_VIDEODRIVER"] = "dummy"

    global env,toolbox,stats
    env = Environment(playermode="ai",
                      player_controller=player_controller(hidden_neurons),
                      speed="fastest",
                      enemymode="static",
                      enemies=s['enemy'],
                      level=2,
                      visuals=False,
                      logs="off")
    mask = (results_pd['enemy']==int(s['enemy'])) & (results_pd['algorithm']==s['algorithm'])
    best_solutions = results_pd[mask].groupby(['enemy','algorithm','iteration'])['hof'].agg(pd.Series.mode).reset_index()['hof']
    best_solutions = best_solutions.apply(string_to_np_array)
    gains = best_solutions.apply(lambda x: individual_gain(x[0]))
    s['gains'] = gains

In [ ]:
box_pd = pd.DataFrame([simulation_loop[i]['gains'] for i in range(len(simulation_loop))]).T
box_pd.columns = ['ssm/5','ssm/6','ssm/7','gm/5','gm/6','gm/7']
plt.figure(figsize=(12,6))
sns.boxplot(box_pd[['ssm/5','gm/5','ssm/6','gm/6','ssm/7','gm/7']],
            palette=['firebrick','bisque','firebrick','bisque','firebrick','bisque'])
sns.stripplot(box_pd[['ssm/5','gm/5','ssm/6','gm/6','ssm/7','gm/7']],
             palette=['black'],size=3)
plt.ylabel('Individual Gain')
plt.xlabel('Experiment Name (Model/Enemy No)')
plt.title('Mean Individual Gain of Best Performing Individuals')
plt.savefig('boxplot.png')

In [ ]:
%%time
hidden_neurons = 10
n_simulation = 1

for s in simulation_loop[:3]:
    simulation(s['enemy'],s['algorithm'],n_simulation,hidden_neurons,s['params'])

In [ ]:
%%time
hidden_neurons = 10
n_simulation = 1

for s in simulation_loop[3:]
    simulation(s['enemy'],s['algorithm'],n_simulation,hidden_neurons,s['params'])